In [24]:
# Ensures compatibility between Python 2 and 3, specifically for the print() function.
from __future__ import print_function
# Provides functions for manipulating dates and times.
import datetime
# Used for serializing and de-serializing Python objects (storing objects in files).
import pickle
# Enables operations on file paths, such as checking if a file exists.
import os.path
# Imports build to interact with Google APIs, here it is used to interact with the Google Calendar API.
from googleapiclient.discovery import build
# Manages the OAuth 2.0 authentication flow.
from google_auth_oauthlib.flow import InstalledAppFlow
# Manages token refresh requests for the Google API authentication.
from google.auth.transport.requests import Request 
# Provides functions to interact with the operating system, such as file operations.
import os
# Library for text-to-speech conversion in Python.
import pyttsx3
# Provides functionality for recognizing spoken language.
import speech_recognition as sr
# Supports time zone conversions.
import pytz
# Allows running system commands from within Python (e.g., opening Notepad).
import subprocess

In [25]:
# The permissions scope to read from Google Calendar API.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]
# Lists for months, days of the week, and possible suffixes for day numbers (like '1st', '2nd', etc.).
MONTHS = [
    "january",
    "february",
    "march",
    "april",
    "may",
    "june",
    "july",
    "august",
    "september",
    "october",
    "november",
    "december",
]
DAYS = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
DAY_EXTENTIONS = ["rd", "th", "st", "nd"]

In [26]:
# Converts the input text into speech using the pyttsx3 engine.
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [27]:
# Captures audio input from the microphone and recognizes it using Google’s speech recognition service.
# Returns the recognized text in lowercase.
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""

        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))

    return said.lower()

In [28]:
def authenticate_google():
    """Authenticates the user using OAuth 2.0 and returns a Google Calendar service object."""
    creds = None
    # Load credentials from file if they exist
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    # Refresh or acquire new credentials if existing ones are invalid or do not exist
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            try:
                flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
                creds = flow.run_local_server(port=0)
            except Exception as e:
                print(f"An error occurred: {e}")
                return None

        # Save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    service = build("calendar", "v3", credentials=creds)
    return service

In [29]:
# Fetches events from the user's Google Calendar for a specific day and converts them to speech.
def get_events(day, service):
    # Call the Calendar API
    date = datetime.datetime.combine(day, datetime.datetime.min.time())
    end_date = datetime.datetime.combine(day, datetime.datetime.max.time())
    utc = pytz.UTC
    date = date.astimezone(utc)
    end_date = end_date.astimezone(utc)

    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=date.isoformat(),
            timeMax=end_date.isoformat(),
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
        speak("No upcoming events found.")
    else:
        speak(f"You have {len(events)} events on this day.")

        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            print(start, event["summary"])
            start_time = str(start.split("T")[1].split("-")[0])
            if int(start_time.split(":")[0]) < 12:
                start_time = start_time + "am"
            else:
                start_time = (
                    str(int(start_time.split(":")[0]) - 12) + start_time.split(":")[1]
                )
                start_time = start_time + "pm"

            speak(event["summary"] + " at " + start_time)

In [30]:
# Analyzes a spoken or written date and extracts the corresponding datetime.date object based on what was said.
def get_date(text):
    text = text.lower()
    today = datetime.date.today()

    if text.count("today") > 0:
        return today

    day = -1
    day_of_week = -1
    month = -1
    year = today.year

    for word in text.split():
        if word in MONTHS:
            month = MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week = DAYS.index(word)
        elif word.isdigit():
            day = int(word)
        else:
            for ext in DAY_EXTENTIONS:
                found = word.find(ext)
                if found > 0:
                    try:
                        day = int(word[:found])
                    except:
                        pass

    if (
        month < today.month and month != -1
    ):  # if the month mentioned is before the current month set the year to the next
        year = year + 1

    if month == -1 and day != -1:  # if we didn't find a month, but we have a day
        if day < today.day:
            month = today.month + 1
        else:
            month = today.month

    # if we only found a day of the week
    if month == -1 and day == -1 and day_of_week != -1:
        current_day_of_week = today.weekday()
        dif = day_of_week - current_day_of_week

        if dif < 0:
            dif += 7
            if text.count("next") >= 1:
                dif += 7

        return today + datetime.timedelta(dif)

    if day != -1:  # FIXED FROM VIDEO
        return datetime.date(month=month, day=day, year=year)

In [31]:
# Saves a note as a text file and opens it in Notepad.
def note(text):
    date = datetime.datetime.now()
    file_name = str(date).replace(":", "-") + "-note.txt"
    with open(file_name, "w") as f:
        f.write(text)

    subprocess.Popen(["notepad.exe", file_name])

In [33]:
# The main loop listens for a specific wake word ("hey aous ")
# and responds to commands related to checking calendar events or making notes based on specific phrases.
WAKE = "hey aous"
SERVICE = authenticate_google()
print("Start")

while True:
    print("Listening")
    text = get_audio()

    if text.count(WAKE) > 0:
        speak("I am ready")
        text = get_audio()

        CALENDAR_STRS = ["what do i have", "do i have plans", "am i busy"]
        for phrase in CALENDAR_STRS:
            if phrase in text:
                date = get_date(text)
                if date:
                    get_events(date, SERVICE)
                else:
                    speak("I don't understand")

        NOTE_STRS = ["make a note", "write this down", "remember this"]
        for phrase in NOTE_STRS:
            if phrase in text:
                speak("What would you like me to write down?")
                note_text = get_audio()
                note(note_text)
                speak("I've made a note of that.")

An error occurred: [Errno 2] No such file or directory: 'credentials.json'
Start
Listening
Exception: 
Listening
Exception: 
Listening
Exception: 
Listening
hello
Listening
Exception: 
Listening
make a note
Listening
write this down
Listening
pleasure to meet you
Listening
do I have a plant
Listening


KeyboardInterrupt: 